# Build a client container

In [ ]:
%%bash

export PROJECT_NAME='YOUR_PROJECT_NAME'
docker build -t locust_tester -f Dockerfile .
docker tag locust_tester gcr.io/${PROJECT_NAME}/locust_tester
docker push gcr.io/${PROJECT_NAME}/locust_tester

# Create a Locust Cluster

In [ ]:
%%writefile service_master.yaml

kind: Service
apiVersion: v1
metadata:
  name: locust-master
  labels:
    app: locust-master
spec:
  ports:
  - name: loc-master
    port: 8089
    targetPort: 8089
    protocol: TCP
  - name: loc-master-p1
    port: 5557
    targetPort: 5557
    protocol: TCP
  - name: loc-master-p2
    port: 5558
    targetPort: 5558
    protocol: TCP
  selector:
    app: locust-master
  type: LoadBalancer

In [ ]:
!kubectl apply -f service_master.yaml

In [ ]:
!kubectl get svc inference-server -o "jsonpath={.spec['clusterIP']}"

In [ ]:
%%writefile deployment_master.yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: locust-master
  labels:
    name: locust-master
spec:
  replicas: 1
  selector:
    matchLabels:
      app: locust-master
  template:
    metadata:
      labels:
        app: locust-master
    spec:
      containers:
      - name: locust-master
        image: gcr.io/sandbox-kathryn-ml/locust_tester
        ports:
          - name: loc-master
            containerPort: 8089
            protocol: TCP
          - name: loc-master-p1
            containerPort: 5557
            protocol: TCP
          - name: loc-master-p2
            containerPort: 5558
            protocol: TCP
        command: ["locust","-f","locust/trtis_grpc_client.py"]
        args: ["--host", "CLUSTER_IP_TRTIS", "--master"]
        resources:
          requests:
            cpu: 200m

In [ ]:
!kubectl get svc locust-master -o "jsonpath={.spec['clusterIP']}"

In [ ]:
%%writefile deployment_slave.yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: locust-slave
  labels:
    name: locust-slave
spec:
  replicas: 3
  selector:
    matchLabels:
      app: locust-slave
  template:
    metadata:
      labels:
        app: locust-slave
    spec:
      containers:
      - name: locust-slave
        image: gcr.io/sandbox-kathryn-ml/locust_tester
        command: ["locust","-f","locust/trtis_grpc_client.py"]
        args: ["--slave", "--master-host=CLUSTER_IP_LOCUST_MASTER"]
        resources:
          requests:
            cpu: 100m

In [ ]:
!kubectl apply -f deployment_master.yaml
!kubectl apply -f deployment_slave.yaml

In [ ]:
!kubectl get svc

In [ ]:
!kubectl get pods